# [모듈 2.2] SageMaker 내장 알고리즘의 모델 훈련
Download | Structure | Preprocessing (Built-in) | **Train Model (Built-in)** (4단계 중의 4/4)
```

```

### [알림] <font coler="red"> conda_mxnet_latest_p37 커널 </font> 과 함께 사용해야 합니다.

* 이 노트북은 `1.1.download_data`, `1.2.structuring_data` 및 `2.1.builtin_preprocessing`으로 시작하는 일련의 노트북의 일부입니다.


<pre>
</pre>

# 노트북 요약
---
이 노트북에서는 SageMaker SDK를 사용하여 SageMaker의 내장 이미지 분류 알고리즘에 대한 Estimator를 생성하고 원격 EC2 인스턴스에서 훈련합니다.

# 1. 환경 설정 
- 카테고리 레이블, 버킷 이름, RecordIO 훈련 및 검증 파일 경로를 로딩 합니다.


In [1]:
import boto3
import shutil
import urllib
import pickle
import pathlib
import tarfile
import subprocess
import sagemaker

In [2]:
%store -r bucket_name
%store -r train_builtin_s3_uri
%store -r val_builtin_s3_uri

In [3]:
with open("pickled_data/category_labels.pickle", "rb") as f:
    category_labels = pickle.load(f)


# 2. 내장 SageMaker 이미지 분류 알고리즘 실행하기
___

## 훈련 및 검증 데이터의 입력 쳐널 정의
- 훈련 및 검증의 경로 및 RecordIO 포맷 , S3 다운로드를 'Pipe" 로 설정 합니다.

In [4]:
train_data = sagemaker.inputs.TrainingInput(
#     s3_data=f"s3://{bucket_name}/data/train",
    s3_data= train_builtin_s3_uri,
    content_type="application/x-recordio",
    s3_data_type="S3Prefix",
    input_mode="Pipe",
)

val_data = sagemaker.inputs.TrainingInput(
#     s3_data=f"s3://{bucket_name}/data/val",
    s3_data=val_builtin_s3_uri,
    content_type="application/x-recordio",
    s3_data_type="S3Prefix",
    input_mode="Pipe",
)

data_channels = {"train": train_data, "validation": val_data}

### 알고리즘의 하이퍼파라미터 구성
- 자세한 내역은 아래를 참조 하세요.
    - https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html
* **num_layers** - 내장된 이미지 분류 알고리즘은 ResNet 아키텍처를 기반으로 합니다. 이 아키텍처에는 사용하는 레이어 수에 따라 다양한 버전이 있습니다. 이 가이드에서는 훈련 속도를 높이기 위해 가장 작은 것을 사용합니다. 알고리즘의 정확도가 정체되어 더 나은 정확도가 필요한 경우 레이어 수를 늘리는 것이 도움이 될 수 있습니다.
* **use_pretrained_model** - 이것은 전이 학습을 위해 사전 훈련된 모델에서 가중치를 초기화합니다. 그렇지 않으면 가중치가 무작위로 초기화됩니다.
* **augmentation_type** - 훈련 세트에 증강을 추가하여 모델을 더 잘 일반화할 수 있습니다. 작은 데이터 세트의 경우 증강은 훈련을 크게 향상시킬 수 있습니다.
* **image_shape** - 모든 이미지의 채널, 높이, 너비
* **num_classes** - 데이터세트의 클래스 수
* **num_training_samples** - 훈련 세트의 총 이미지 수(진행률 계산에 사용)
* **mini_batch_size** - 훈련 중에 사용할 배치 크기입니다.
* **에포크** - 에포크는 훈련 세트의 한 주기를 나타내며 훈련할 에포크가 많다는 것은 정확도를 향상시킬 기회가 더 많다는 것을 의미합니다. 적절한 값은 시간과 예산 제약에 따라 5~25 Epoch 범위입니다. 이상적으로는 검증 정확도가 안정되기 직전에 올바른 Epoch 수가 있습니다.
* **learning_rate**: 훈련의 각 배치 후에 우리는 해당 배치에 대해 가능한 최상의 결과를 제공하기 위해 모델의 가중치를 업데이트합니다. 학습률은 가중치를 업데이트해야 하는 정도를 제어합니다. 모범 사례는 0.2에서 .001 사이의 값을 지정하며 일반적으로 1보다 높지 않습니다. 학습률이 높을수록 훈련이 최적의 가중치로 더 빨리 수렴되지만 너무 빠르면 목표를 초과할 수 있습니다. 이 예에서는 사전 훈련된 모델의 가중치를 사용하므로 가중치가 이미 최적화되어 있고 가중치에서 너무 멀리 이동하고 싶지 않기 때문에 더 낮은 학습률로 시작하려고 합니다.
* **precision_dtype** - 모델의 가중치에 대해 32비트 부동 데이터 유형을 사용할지 16비트를 사용할지 여부. 메모리 관리 문제가 있는 경우 16비트를 사용할 수 있습니다. 그러나 가중치는 빠르게 증가하거나 줄어들 수 있으므로 32비트 가중치를 사용하면 이러한 문제에 대한 훈련이 더욱 강력해지며 일반적으로 대부분의 프레임워크에서 기본값입니다.

In [5]:
num_classes = len(category_labels)
num_training_samples = len(set(pathlib.Path("data_structured/train").rglob("*.jpg")))

In [6]:
hyperparameters = {
    "num_layers": 18,
    "use_pretrained_model": 1, # Pretrained Model (ResNet) 의 가중치 가져오기
    "augmentation_type": "crop_color_transform",
    "image_shape": "3,224,224",
    "num_classes": num_classes,
    "num_training_samples": num_training_samples,
    "mini_batch_size": 64,
    "epochs": 5,
    "learning_rate": 0.001,
    "precision_dtype": "float32",
}

## 내장 알고리즘의 Docker 경로 및 컨피그 설정

In [7]:
training_image = sagemaker.image_uris.retrieve(
    "image-classification", sagemaker.Session().boto_region_name
)

In [8]:
algo_config = {
    "hyperparameters": hyperparameters,
    "image_uri": training_image,
    "role": sagemaker.get_execution_role(),
    "instance_count": 1,
    "instance_type": "ml.p3.2xlarge",
    "volume_size": 100,
    "max_run": 360000,
    "output_path": f"s3://{bucket_name}/data/output",
}

## 알고리즘 Estimator 를 생성하고 훈련 시작

In [9]:
algorithm = sagemaker.estimator.Estimator(**algo_config)

In [10]:
algorithm.fit(inputs=data_channels, logs=True)

2022-01-03 09:10:18 Starting - Starting the training job...
2022-01-03 09:10:19 Starting - Launching requested ML instancesProfilerReport-1641201017: InProgress
...
2022-01-03 09:11:15 Starting - Preparing the instances for training............
2022-01-03 09:13:09 Downloading - Downloading input data
2022-01-03 09:13:09 Training - Downloading the training image......
2022-01-03 09:14:09 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
[01/03/2022 09:14:13 INFO 140191411951424] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/image_classification/default-input.json: {'use_pretrained_model': 0, 'num_layers': 152, 'epochs': 30, 'learning_rate': 0.1, 'lr_scheduler_factor': 0.1, 'optimizer': 'sgd', 'momentum': 0, 'weight_decay': 0.0001, 'beta_1': 0.9, 'beta_2': 0.999, 'eps': 1e-08, 'gamma': 0.9, 'mini_batch_size': 32, 'image_shape': '3,224,224', 'precision_dtype': 'float32'}
[01/03/2022 09:14:13 INF

# 3. 훈련 결과 이해하기
___

```
[09/14/2020 05:37:38 INFO 139869866030912] Epoch[0] Batch [20]#011Speed: 111.811 samples/sec#011accuracy=0.452381
[09/14/2020 05:37:54 INFO 139869866030912] Epoch[0] Batch [40]#011Speed: 131.393 samples/sec#011accuracy=0.570503
[09/14/2020 05:38:10 INFO 139869866030912] Epoch[0] Batch [60]#011Speed: 139.540 samples/sec#011accuracy=0.617700
[09/14/2020 05:38:27 INFO 139869866030912] Epoch[0] Batch [80]#011Speed: 144.003 samples/sec#011accuracy=0.644483
[09/14/2020 05:38:43 INFO 139869866030912] Epoch[0] Batch [100]#011Speed: 146.600 samples/sec#011accuracy=0.664991
```

훈련이 시작되었습니다:
* Epoch[0]: 하나의 Epoch는 모든 데이터를 통한 하나의 훈련 주기에 해당합니다. SGD 및 Adam과 같은 확률적 최적화 프로그램은 여러 에포크를 실행하여 정확도를 개선합니다. 
- 훈련 알고리즘이 랜덤 데이터 증강으로 인해 수정된 이미지 대해서 매 에포크마다 훈련을 수행 합니다. 
    - 즉 에포크마다 데이터 증강으로 인한 새로운 이미지가를 훈련한다는 의미 입니다.
* 배치: 훈련 알고리즘에 의해 처리된 배치의 수. 우리는 `mini_batch_size` 하이퍼파라미터에서 하나의 배치를 64개의 이미지로 지정했습니다. SGD와 같은 알고리즘의 경우 모델은 배치마다 자체적으로 업데이트할 기회를 얻습니다.
* 속도: 초당 훈련 알고리즘으로 전송되는 이미지의 수입니다. 이 정보는 데이터 세트의 변경 사항이 훈련 속도에 미치는 영향을 결정하는 데 중요합니다.
* 정확도: 각 간격(이 경우 20개 배치)에서 달성한 훈련 정확도.

```

[09/14/2020 05:38:58 INFO 139869866030912] Epoch[0] Train-accuracy=0.677083
[09/14/2020 05:38:58 INFO 139869866030912] Epoch[0] Time cost=102.745
[09/14/2020 05:39:02 INFO 139869866030912] Epoch[0] Validation-accuracy=0.729492
[09/14/2020 05:39:02 INFO 139869866030912] Storing the best model with validation accuracy: 0.729492
[09/14/2020 05:39:02 INFO 139869866030912] Saved checkpoint to "/opt/ml/model/image-classification-0001.params"
```

- 훈련의 첫 번째 에포크가 종료 되었습니다. (이 예에서는 한 에포크에 대해서만 훈련합니다). 
- 최종 훈련 정확도와 검증 세트의 정확도가 보고됩니다. 이 두 숫자를 비교하는 것은 모델이 과적합인지 과소적합인지와 Bias/Varience 트레이드오프를 결정하는 데 중요합니다. 저장할 모델은 검증 데이터 세트의 정확도가 가장 큰 가중치를 사용합니다.

```

2020-09-14 05:39:03 Uploading - Uploading generated training model
2020-09-14 05:39:15 Completed - Training job completed
Training seconds: 235
Billable seconds: 235
```

최종 모델 가중치는 'algo_config'의 'output_path' 의 지정된 S3의 '.tar.gz'로 저장됩니다. EC2 인스턴스가 데이터에 대해 훈련하는 시간에 대해서만 비용이 청구되기 때문에 총 청구 가능 시간(초)도 보고되어 훈련 비용을 계산하는 데 도움이 됩니다. 

## 다음 과정
이것으로 SageMaker의 내장 알고리즘에 대한 이미지 데이터 준비 가이드를 마칩니다. 모델을 배포하고 테스트 데이터에 대한 예측을 얻으려면 여기에서 찾을 수 있습니다.
- [추론을 위한 모델 배포](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)